In [14]:
import gensim
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
import tqdm
from utils import *
import plotly.graph_objects as go

In [2]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Load Data

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [4]:
train_df.columns

Index(['class', 'title', 'description'], dtype='object')

In [5]:
for index, row in train_df.iterrows():
    if row['class'] == 1:
        train_df.at[index, 'topic'] = 'World'
    elif row['class'] == 2:
        train_df.at[index, 'topic'] = 'Sports'
    elif row['class'] == 3:
        train_df.at[index, 'topic'] = 'Business'
    else:
        train_df.at[index, 'topic'] = 'Sci/Tech'

In [6]:
for index, row in test_df.iterrows():
    if row['class'] == 1:
        test_df.at[index, 'topic'] = 'World'
    elif row['class'] == 2:
        test_df.at[index, 'topic'] = 'Sports'
    elif row['class'] == 3:
        test_df.at[index, 'topic'] = 'Business'
    else:
        test_df.at[index, 'topic'] = 'Sci/Tech'

In [7]:
# # Plots to check distribution of classes in the datasets
# fig_train = px.bar(train_df, x=train_df['topic'].value_counts().index, y=train_df['topic'].value_counts().values, color=train_df['topic'].value_counts().index, 
#                    title='Distribution of topics in training dataset', width=1100, height=500,
#                    labels={
#                        'x':'Topic', 'y':'Number of News Articles','color':'Topic'
#                    })
# fig_train.show()
# fig_train.write_image('../results/EDA/topic-dist-train.pdf', engine='kaleido')

# fig_test = px.bar(test_df, x=test_df['topic'].value_counts().index, y=test_df['topic'].value_counts().values, color=test_df['topic'].value_counts().index, 
#                    title='Distribution of topics in testing dataset', width=1100, height=500,
#                    labels={
#                        'x':'Topic', 'y':'Number of News Articles','color':'Topic'
#                    })
# fig_test.show()
# fig_test.write_image('../results/EDA/topic-dist-test.pdf', engine='kaleido')

In [8]:
# Checking the average length of news article for each class
train_avg_len_by_class = (train_df.groupby('topic')['description'].apply(lambda x: np.mean(x.str.len())).reset_index(name='avg_news_len'))
test_avg_len_by_class = (test_df.groupby('topic')['description'].apply(lambda x: np.mean(x.str.len())).reset_index(name='avg_news_len'))

In [9]:
train_avg_len_by_class

,topic,avg_news_len
0,Business,198.690267
1,Sci/Tech,193.852533
2,Sports,185.171367
3,World,195.893900


In [10]:
test_avg_len_by_class

,topic,avg_news_len
0,Business,197.831053
1,Sci/Tech,193.951579
2,Sports,183.838947
3,World,194.010526


In [11]:
# Preprocess the news description
tqdm.tqdm.pandas()
train_df['news_tokenized'] = train_df['description'].progress_apply(lambda x: preprocess_text(str(x)))
test_df['news_tokenized'] = test_df['description'].progress_apply(lambda x: preprocess_text(str(x)))

100%|██████████| 7600/7600 [00:08<00:00, 877.68it/s]


## Feature Representations

### TF-IDF

In [12]:
ag_news_df = pd.concat([train_df, test_df], ignore_index=True)

In [13]:
tf_idfvectorizer = TfidfVectorizer(strip_accents='unicode', stop_words='english', min_df=2)
x_tfidf = tf_idfvectorizer.fit_transform(ag_news_df.news_tokenized)

In [14]:
performance_metrics = pd.DataFrame(columns=['feature-extraction','run#', 'epoch', 'state', 'AMI','ARI','NMI'])

In [15]:
for run in range(1, 11, 1):
    print('Run #', run)
    for epoch, state in zip(range(25, 300, 15), range(2, 40, 2)):
        k_means = KMeans(n_clusters=4, init='k-means++', max_iter=epoch, random_state=state)
        k_means.fit(x_tfidf)
        
        ami = metrics.adjusted_mutual_info_score(ag_news_df['class'], k_means.labels_)
        ari = metrics.adjusted_rand_score(ag_news_df['class'], k_means.labels_)
        nmi = metrics.normalized_mutual_info_score(ag_news_df['class'], k_means.labels_)

        # print(run, epoch, state)
        
        performance_metrics = performance_metrics.append({'feature-extraction':'tf-idf', 'run#':run, 'epoch':epoch, 'state':state, 'AMI':ami, 'ARI': ari, 'NMI':nmi}, ignore_index=True)

Run # 1
Run # 2
Run # 3
Run # 4
Run # 5
Run # 6
Run # 7
Run # 8
Run # 9
Run # 10


In [16]:
performance_metrics

,feature-extraction,run#,epoch,state,AMI,ARI,NMI
0,tf-idf,1,25,2,0.205560,0.134475,0.205585
1,tf-idf,1,40,4,0.229895,0.164062,0.229920
2,tf-idf,1,55,6,0.230795,0.165290,0.230819
3,tf-idf,1,70,8,0.301438,0.232790,0.301458
4,tf-idf,1,85,10,0.356919,0.266382,0.356937
...,...,...,...,...,...,...,...
185,tf-idf,10,235,30,0.235055,0.120214,0.235081
186,tf-idf,10,250,32,0.246248,0.137647,0.246272
187,tf-idf,10,265,34,0.230397,0.164805,0.230422
188,tf-idf,10,280,36,0.229908,0.164073,0.229933


In [18]:
performance_metrics.to_csv('performance_tfidf_kmeans.csv', index=False)

In [3]:
performance_metrics = pd.read_csv('../results/Finding optimal epochs/performance_tfidf_kmeans.csv')

In [24]:
mean_performance = performance_metrics.groupby('epoch', as_index=False)[['AMI','ARI','NMI']].mean()

In [26]:
mean_performance.to_csv('../results/Finding optimal epochs/mean_performance-tfidf_kmeans.csv', index=False)